In [1]:
# !git clone "https://github.com/axchizhov/masked_autoencoder"

In [2]:
# %cd masked_autoencoder
# !pip install -e .

## Hyperparameters

In [2]:
BATCH_SIZE = 256
LR = 1e-5
BETAS = (0.9, 0.95)
NUM_EPOCHS = 100

IMAGE_SIZE = 32
PATCH_SIZE = 8
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2
IN_CHANS = 3

ENCODER_EMBED_DIM = 128
ENCODER_NUM_HEADS = 4
ENCODER_DEPTH = 6

DECODER_EMBED_DIM = 64
DECODR_NUM_HEADS = 4
DECODER_DEPTH = 2

SEED = 42

# Preapre data

In [3]:
import torch
import torchvision.transforms as T

from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

torch.manual_seed(SEED)

# TODO: fix dataset loading
# TODO: add augmentations

train_data = CIFAR10('../data/', train=True, download=True, transform=T.ToTensor())
# train_data = train_data[:40000]

valid_data = CIFAR10('../data/', train=True, download=True, transform=T.ToTensor())
# valid_data = valid_data[40000:]

test_data = CIFAR10('../data/', train=False, download=True, transform=T.ToTensor())

train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
valid_dataloader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Train the autoencoder

In [ ]:
from torch import optim
from src.plots import grid_plot, plot_reconstructed
from torch.utils.tensorboard import SummaryWriter

from src.masked_autoencoder import MaskedAutoencoder


net = MaskedAutoencoder(
    img_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    in_chans=IN_CHANS,
    encoder_embed_dim=ENCODER_EMBED_DIM,
    encoder_num_heads=ENCODER_NUM_HEADS,
    encoder_depth=ENCODER_DEPTH,
    decoder_embed_dim=DECODER_EMBED_DIM,
    decoder_num_heads=DECODR_NUM_HEADS,
    decoder_depth=DECODER_DEPTH,
)

optimizer = optim.AdamW(net.parameters(), lr=LR, betas=BETAS)
scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=10)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

# Set optimization scheduler
# total_steps = int((len(train_dataloader) / batch_size) * num_epochs)
# warmup_epoch_percentage = 0.15
# warmup_steps = int(total_steps * warmup_epoch_percentage)
# warmup_steps

# scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, warmup_steps)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)

log_writer = SummaryWriter()

net.train()
step = 1
for epoch in range(NUM_EPOCHS):
    for image_batch, _ in train_dataloader:
        image_batch = image_batch.to(device)
        optimizer.zero_grad()
        loss, pred, mask = net(image_batch)
        loss.backward()
        optimizer.step()
        
        log_writer.add_scalar("Loss/train", loss.item(), step)
        
        # if step % 20 == 0:
        step += 1
    
    scheduler.step()
    
    print(epoch)
    
    reconstructed = net.unpatchify(pred)
    plot_reconstructed(image_batch, reconstructed)
    grid = grid_plot(image_batch, reconstructed)
    log_writer.add_image('Original vs Reconstructed', grid, step)


# log_writer.add_graph(net, batch)
log_writer.close()

In [ ]:
# # evaluate accuracy after training
# model.eval()
# y_pred = model(X_test)
# acc = (y_pred.round() == y_test).float().mean()
# acc = float(acc)
# print("Model accuracy: %.2f%%" % (acc*100))

In [11]:
# %tensorboard --logdir runs/

UsageError: Line magic function `%tensorboard` not found.
